In [27]:
import pandas as pd
import requests
import gmaps
from config import g_key
import gmaps.datasets

In [28]:
gmaps.configure(api_key=g_key)

In [29]:
#load file
file_to_load = "Path/WeatherPy_vacation.csv"

In [30]:
file =pd.read_csv(file_to_load)

In [31]:
#create a new DataFrame
itinerary_df = pd.DataFrame(file)

In [32]:
itinerary_df.drop(itinerary_df.filter(regex="Unname"),axis=1, inplace=True)

In [33]:
itinerary_df

,City,Country,Max Temp,Lat,Lng,Weather Description,Hotel Name
0,Fairbanks,US,25.00,64.84,-147.72,light snow,Westmark Fairbanks Hotel & Conference Center
1,Husavik,IS,28.40,66.04,-17.34,broken clouds,Bjarnabúð
2,Yarmolyntsi,UA,30.00,49.19,26.84,broken clouds,NaN
3,Kodiak,US,33.80,57.79,-152.41,clear sky,Mission Lake Bed and Breakfast
4,Albany,US,33.01,42.60,-73.97,overcast clouds,NaN
...,...,...,...,...,...,...,...
70,Hasaki,JP,24.80,35.73,140.83,few clouds,Inubosaki Kanko Hotel
71,Morden,CA,23.00,49.19,-98.10,few clouds,Morden Motor Inn
72,Bismil,TR,23.00,37.85,40.67,scattered clouds,Dostlar Otomotiv
73,Spearfish,US,30.00,44.49,-103.86,clear sky,Travelodge by Wyndham Spearfish


In [34]:
#select four cities in close proximity on the map,  create separate DataFrames for each city
city1 = itinerary_df[itinerary_df['City']=='Husavik']
city1 = pd.DataFrame(city1)
city2 = itinerary_df[itinerary_df['City']=='Akureyri']
city2 = pd.DataFrame(city2)
city3 = itinerary_df[itinerary_df['City']=='Borgarnes']
city3 = pd.DataFrame(city3)
city4 = itinerary_df[itinerary_df['City']=='Olafsvik']
city4 = pd.DataFrame(city4)

In [35]:
#Use the list indexing to get the latitude-longitude pairs for each city DataFrame as tuples
husavik = list(city1[['Lat', 'Lng']].itertuples(index=False, name=None))
akureyri =list(city2[['Lat', 'Lng']].itertuples(index=False, name=None))
borgarnes = list(city3[['Lat', 'Lng']].itertuples(index=False, name=None))
olafsvik = list(city4[['Lat', 'Lng']].itertuples(index=False, name=None))


In [40]:
husavik = (66.04, -17.34)
akureyri = (65.68, -18.09)
borgarnes = (64.54, -21.92)
olafsvik = (64.89, -23.71)
#Create a marker layer map for the four cities
fig = gmaps.figure()
icetrip = gmaps.directions_layer(
        husavik, akureyri, waypoints=[borgarnes, olafsvik], travel_mode = 'DRIVING')
fig.add_layer(icetrip)
fig

Figure(layout=FigureLayout(height='420px'))

In [37]:
#Create a new DataFrame that has all the individual city DataFrames 
cities_df = pd.concat([city1, city2, city3, city4], axis=0)
cities_df

,City,Country,Max Temp,Lat,Lng,Weather Description,Hotel Name
1,Husavik,IS,28.40,66.04,-17.34,broken clouds,Bjarnabúð
16,Akureyri,IS,28.40,65.68,-18.09,broken clouds,Icelandair Hotel Akureyri
32,Borgarnes,IS,27.00,64.54,-21.92,clear sky,Hotel Hafnarfjall
38,Olafsvik,IS,26.22,64.89,-23.71,overcast clouds,Hotel Olafsvik


In [38]:
#create a pop-up marker
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Weather Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

hotel_info = [info_box_template.format(**row) for index, row in cities_df.iterrows()]

In [39]:
locations = cities_df[["Lat", "Lng"]]
max_temp = cities_df["Max Temp"]
weather_desc = cities_df["Weather Description"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))